In [1]:
from splinter import Browser
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time


from bs4 import BeautifulSoup


In [2]:
# Set up Chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--start-maximized")

In [3]:
# Initialize the browser
service = Service(ChromeDriverManager().install())
browser = Browser('chrome', service=service, options=chrome_options)

In [4]:
# Visit LinkedIn
url = "https://www.linkedin.com/feed/"
browser.visit(url)

# Wait for page to load
time.sleep(3)

In [5]:
# Get page title
page_title = browser.title
print(f"Page Title: {page_title}")


Page Title: LinkedIn Login, Sign in | LinkedIn


In [6]:


# If the login page appears, enter credentials
if "LinkedIn Login" in page_title or "Sign in | LinkedIn" in page_title:
    print("Login page detected. Entering credentials...")

    # Enter username
    browser.fill("session_key", "")  # Replace with your email

    # Enter password
    browser.fill("session_password", "")  # Replace with your password

    # Find and click the Sign In button using its class
    sign_in_button = browser.find_by_css("button.btn__primary--large").first
    sign_in_button.click()

    # Wait for login
    time.sleep(5)

    # Check if login was successful
    print(f"New Page Title: {browser.title}")

else:
    print("Already logged in!")




Login page detected. Entering credentials...
New Page Title: (8) Feed | LinkedIn


In [7]:
# Locate the search input field and enter a search term
search_input = browser.find_by_css("input.search-global-typeahead__input").first
search_input.fill("Data Engineer")

# Press Enter to submit the search
search_input.type("\n")

'\n'

In [8]:
# Wait to ensure the search results page loads
time.sleep(3)

# Find and click the "Jobs" filter button
jobs_button = browser.find_by_css("button.search-reusables__filter-pill-button").first
jobs_button.click()

# Wait for the jobs page to load
time.sleep(5)

In [9]:
# Wait for the jobs filter page to load
time.sleep(3)

# Locate the location input field using a wildcard in XPath
location_input = browser.find_by_xpath("//*[starts-with(@id, 'jobs-search-box-location-id-ember')]").first

# Fill in "Tokyo" as the location
if location_input:
    location_input.fill("Tokyo")
    print("Location set to Tokyo")
else:
    print("Location input field not found")

# Wait briefly after filling in the location
time.sleep(2)

# Click the "Search" button to apply the location filter
search_button = browser.find_by_css("button.jobs-search-box__submit-button").first

if search_button:
    search_button.click()
    print("Clicked the search button.")
else:
    print("Search button not found.")

# Wait for results to update
time.sleep(5)




Location set to Tokyo
Clicked the search button.


In [10]:
page_html = browser.html
soup = BeautifulSoup(page_html, "html.parser")

In [11]:
soup

<html class="theme theme--mercado app-loader--default artdeco windows" lang="en"><head>
<script async="" nonce="" src="https://www.googletagmanager.com/gtag/js?id=AW-851212376&amp;l=dataLayer&amp;cx=c&amp;gtm=453e5240za200" type="text/javascript"></script><script async="" nonce="" src="https://www.googletagmanager.com/gtag/js?id=DC-9261636&amp;l=dataLayer&amp;cx=c&amp;gtm=453e5240za200" type="text/javascript"></script><script async="" nonce="" src="https://www.googletagmanager.com/gtag/js?id=DC-10880300&amp;l=dataLayer&amp;cx=c&amp;gtm=453e5240za200" type="text/javascript"></script><script async="" nonce="" src="https://www.googletagmanager.com/gtag/js?id=AW-867846157&amp;l=dataLayer&amp;cx=c&amp;gtm=453e5240za200" type="text/javascript"></script><script async="" charset="utf-8" id="utag_234" src="https://www.googletagmanager.com/gtag/js" type="text/javascript"></script><script nonce="">!function(i,n){void 0!==i.addEventListener&&void 0!==i.hidden&&(n.liVisibilityChangeListener=functio

In [12]:

from bs4 import BeautifulSoup
import time
import pandas as pd

all_jobs = []  # Store job listings from all pages

# Wait for pagination buttons to load
time.sleep(3)
page_html = browser.html
soup = BeautifulSoup(page_html, "html.parser")

# Find all pagination buttons dynamically
pagination_buttons = browser.find_by_css("button[aria-label^='Page ']")
total_pages = len(pagination_buttons)  # Count total pages

print(f"Total pages found: {total_pages}")

# Extract the available page numbers in order
page_numbers = []
for button in pagination_buttons:
    try:
        page_number = int(button.text.strip())  # Extract page number
        page_numbers.append(page_number)
    except ValueError:
        continue  # Skip non-numeric buttons

page_numbers = sorted(set(page_numbers))  # Ensure unique & sorted

print(f"Pages to scrape: {page_numbers}")

# Iterate through all valid pages
for page_num in page_numbers:
    print(f"Scraping page {page_num}")

    # Wait for jobs to load
    time.sleep(3)

    # Get the current page's HTML
    page_html = browser.html
    soup = BeautifulSoup(page_html, "html.parser")

    # Find all job listings directly
    job_listings = soup.find_all("li", {"data-occludable-job-id": True})

    print(f"Total jobs found on page {page_num}: {len(job_listings)}")

    # Extract job titles and company names
    for job in job_listings:
        print(f'Job{job}')
        try:
            # Extract job title
            job_title_element = job.find("a", class_="job-card-container__link")
            job_title = job_title_element.get_text(strip=True) if job_title_element else "No Title Found"

            # Extract company name
            company_element = job.find("span", class_="artdeco-entity-lockup__subtitle")
            company = company_element.get_text(strip=True) if company_element else "No Company Found"

            # Store only jobs that have a title
            if job_title != "No Title Found":
                all_jobs.append((job_title, company))

        except Exception as e:
            print(f"Error extracting job: {e}")

    # If there's another page, click the corresponding button
    if page_num < max(page_numbers):
        try:
            next_page_button = browser.find_by_css(f"button[aria-label='Page {page_num + 1}']").first
            if next_page_button:
                print(f"Clicking Page {page_num + 1} button...")
                next_page_button.click()
                time.sleep(3)  # Wait for new page to load
            else:
                print(f"Page {page_num + 1} button not found. Stopping.")
                break
        except:
            print(f"Error clicking Page {page_num + 1}.")
            break

print("Scraping completed!")

# Print all collected jobs
for job_title, company in all_jobs:
    print(f"Job Title: {job_title} | Company: {company}")

print(f"Total jobs collected: {len(all_jobs)}")

# Save results to CSV
df = pd.DataFrame(all_jobs, columns=["Job Title", "Company"])
df.to_csv("linkedin_jobs.csv", index=False)
print("Saved to linkedin_jobs.csv")



Total pages found: 5
Pages to scrape: [1, 2, 3, 4, 5]
Scraping page 1
Total jobs found on page 1: 25
Job<li class="ember-view erfhlCczvjhljBkvmzpmhFlCWLeFmsKTIwhtA occludable-update p0 relative scaffold-layout__list-item" data-occludable-job-id="4139693939" id="ember812">
<div>
<div aria-current="page" class="display-flex job-card-container relative job-card-list job-card-container--clickable job-card-list--underline-title-on-hover jobs-search-results-list__list-item--active jobs-search-two-pane__job-card-container--viewport-tracking-0" data-job-id="4139693939" data-view-name="job-card">
<div>
<div class="job-card-list__entity-lockup artdeco-entity-lockup artdeco-entity-lockup--size-4 ember-view" id="ember813">
<div class="job-card-list__logo artdeco-entity-lockup__image artdeco-entity-lockup__image--type-square ember-view" id="ember814" type="square">
<div class="ivm-image-view-model job-card-list__logo-ivm">
<div class="ivm-view-attr__img-wrapper">
<!-- -->
<!-- --> <img alt="Quantex

In [13]:
# Close the browser
browser.quit()